# Pulsar Prediction

## Introduction

### Background
Pulsars are a rare type of Neutron star which look like a “flickering star”. It flickers because it will produce two steady, narrow beams as it spins. They can produce radio emission detectable on Earth.

The extreme state matter of a pulsar can be used to find planets beyond our solar system, and measure the cosmic distance. Gravitational waves are outcomes from pulsars that could be evidence about energetic cosmic events like enormous black holes’ collision (Cofield, 2016).

A common method of finding a pulsar is radio telescopes. However, recently machine learning tools are now being used to automatically label pulsar candidates to facilitate rapid analysis.

### Question
Using the K-nearest neighbors algorithm, to what accuracy can we predict whether an observed star is a pulsar?

### Dataset
We are working with a Kaggle dataset which describes attributes about stars, along with labels of being pulsar stars or not. Since the testing dataset is not labelled, we are unable to use it for testing.


## Preliminary exploratory data analysis

### Importing libraries

In [ ]:
suppressMessages(library(tidyverse))
suppressMessages(library(repr))
suppressMessages(library(tidymodels))
options(repr.matrix.max.rows = 6)

### Reading data
Data was downloaded as CSV files. 
Kaggle provided training and testing data. It should be noted that Kaggle testing
data is unlabelled, so model evaluation must be done with some proportion of training
data.

In [ ]:
pulsar_training = read_csv("https://raw.githubusercontent.com/FrankWhoee/pulsar/master/pulsar_data_train.csv",col_types = cols())

print(paste("pulsar_training has", nrow(pulsar_training), "observations,"))
print(paste(ncol(pulsar_training), "variables, and"))
print(paste(sum(rowSums(is.na(pulsar_training)) > 0), "NA observations."))
print("Column names before shortening are: ")
for(n in names(pulsar_training)){
    print(n)
}

### Shortening column names

In [ ]:
pulsar_names = c()
for(n in names(pulsar_training)){
    acroynm = ''
    for(word in str_split(n, ' ')){
        acroynm = paste(acroynm,substr(word,1,1), sep = '', collapse='')
    }
    pulsar_names = c(pulsar_names,acroynm)
}
names(pulsar_training)<-pulsar_names
print("Column names after shortening are: ")
for(n in names(pulsar_training)){
    print(n)
}

### Removing NA entries

In [ ]:
pulsar_training = na.omit(pulsar_training)
print(paste("pulsar_training has", nrow(pulsar_training), "observations,"))
print(paste(ncol(pulsar_training), "variables, and"))
print(paste(sum(rowSums(is.na(pulsar_training)) > 0), "NA observations."))
pulsar_training

### Data summary and visualization

In [ ]:
class_distribution <- pulsar_training %>%
ggplot(aes(x = t)) +
geom_bar()

class_distribution

We will split the data into positive and negative cases for a more accurate depiction, since there are approximately 10 times more negative cases.

In [ ]:
# Data split
pulsar_pos = pulsar_training %>%
filter(t == 1)

pulsar_neg = pulsar_training %>%
filter(t == 0)

In [ ]:
# Create plots
pulsar_plot = gather(pulsar_training) %>%
filter(key != 't') %>%
ggplot(aes(value)) +
geom_histogram(bins=15, fill='darkblue')+
facet_wrap(~key, scales = 'free_x')

pulsar_pos_plot = gather(pulsar_pos) %>%
filter(key != 't') %>%
ggplot(aes(value)) +
geom_histogram(bins=15, fill='coral')+
facet_wrap(~key, scales = 'free_x')

pulsar_neg_plot = gather(pulsar_neg) %>%
filter(key != 't') %>%
ggplot(aes(value)) +
geom_histogram(bins=15, fill='dodgerblue')+
facet_wrap(~key, scales = 'free_x')

#### Overall plot

We can see that EkotDc, Motip, and Sdotip have roughly normal distributions, but the rest are all skewed to the right.

In [ ]:
pulsar_plot

#### Negative plots

Due to the proportion of negative data, the overall plot and negative-only plot visualizations look identical.

In [ ]:
pulsar_neg_plot

#### Positive plots

The positive plots are dramatically different, however. Most notable are Ekotip, Motip, and SdotDc on account of their different skews and variances.

In [ ]:
pulsar_pos_plot

#### Comparison of statistical attributes

In [ ]:
# Part 1 - Getting means and stdev

posmeans = pulsar_pos %>% 
map_df(mean) %>%
select(-t) %>%
pivot_longer(cols=Motip:SotDc)
names(posmeans) <- c("name", "pos_mean")

posdev = pulsar_pos %>% 
map_df(sd) %>%
select(-t) %>%
pivot_longer(cols=Motip:SotDc) %>%
select(-name)
names(posdev) <- c("pos_stdev")

negmeans = pulsar_neg %>% 
map_df(mean) %>%
select(-t) %>%
pivot_longer(cols=Motip:SotDc)%>%
select(-name)
names(negmeans) <- c("neg_mean")

negdev = pulsar_neg %>% 
map_df(sd) %>%
select(-t) %>%
pivot_longer(cols=Motip:SotDc) %>%
select(-name)
names(negdev) <- c("neg_stdev")

summarized = cbind(posmeans,posdev, negmeans,negdev)
summarized %>% head(4)
summarized %>% tail(4)

## Method

#### Setting Seed

In [ ]:
set.seed(2021)

## Data Prep

In [ ]:
pulsar_training <- mutate (pulsar_training, t = as.factor(t))

split_data <- initial_split(pulsar_training, prop=0.75, strata=t)

training_data <- training(split_data)
testing_data  <- testing(split_data)


## Temporary Recipe Creation

In [ ]:
rec <- recipe(t ~ ., data = training_data) %>%
step_upsample(t, over_ratio = 1, skip = FALSE) %>%
step_scale(all_predictors()) %>%
step_center(all_predictors()) %>%
prep()

cooked_training <- bake(rec, training_data)
cooked_training

### Feature Selection
To choose which features will be selected, we are using ANOVA (Mumtaz, 2020).

In [ ]:
cooked_training <- mutate(cooked_training, t = as.numeric(t))

aov.models = cooked_training %>%
  map(~ summary(aov(cooked_training$t ~ .x))[[1]][["F value"]][1])

as_tibble(aov.models) %>%
pivot_longer(cols=Motip:t, names_to="predictor") %>%
filter(predictor != 't') %>%
arrange(desc(value)) %>%
head(5)

summary(aov(t ~ Motip + Ekotip + SdotDc + EkotDc + Sotip, data = cooked_training))


### New Recipe Creation

In [ ]:
final_training <- select(training_data, Motip , Ekotip , SdotDc , EkotDc , Sotip, t)
final_testing <- select(testing_data, Motip , Ekotip , SdotDc , EkotDc , Sotip, t)

ups_rec <- recipe(t ~ . , data = final_training) %>%
step_upsample(t, over_ratio = 0.5, skip = FALSE) %>%
prep()

final_training <- bake(ups_rec, final_training)
final_testing <- bake(ups_rec, final_testing)

final_rec <- recipe(t ~ . , data = final_training) %>%
step_scale(all_predictors()) %>%
step_center(all_predictors())

final_rec

## Perform 5-fold cross-validation

In [ ]:
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>%
       set_engine("kknn") %>%
       set_mode("classification")

crossv = vfold_cv(final_training, v = 5, strata = t)

k_vals <- tibble(neighbors = seq(from = 1, to = 20 , by = 1))

wf <- workflow() %>%
       add_recipe(final_rec) %>%
       add_model(knn_spec) %>%
       tune_grid(resamples = crossv, grid = k_vals) 
wf_metrics <- collect_metrics(wf)

In [ ]:
options(repr.plot.height = 5, repr.plot.width = 6)

wf_metrics
cross_val_plot <- wf_metrics %>% 
filter(.metric == "accuracy") %>%
ggplot(aes(x=neighbors,y=mean)) +
geom_point()+
geom_line()+
ylab("Accuracy")


cross_val_plot
wfm2 = wf_metrics %>% 
filter(.metric == "accuracy")
max(wfm2$mean)

chosen_k = wf_metrics %>% 
filter(.metric == "accuracy") %>%
arrange(mean) %>%
tail(1) %>%
select(neighbors) %>%
pull()

chosen_k

## Build Model

In [ ]:
# your code here
final_spec = nearest_neighbor(weight_func = "rectangular", neighbors = chosen_k) %>%
       set_engine("kknn") %>%
       set_mode("classification")

final_fit = workflow() %>%
       add_recipe(final_rec) %>%
       add_model(final_spec) %>%
        fit(data = final_training)

In [ ]:
predictions = predict(final_fit, final_testing) %>%
bind_cols(final_testing)

final_metrics = metrics(predictions, .pred_class, t)
final_metrics
final_conf_mat = conf_mat(predictions, .pred_class,t)
final_conf_mat

### Data Analysis

First, we will create a recipe to preprocess our data.

As our dataset is very imbalanced, we want to rebalance the data in a way that can eliminate a falsely high accuracy rate for our model. We will do this by oversampling the positive cases.

We will use the K-nearest neighbors algorithm to classify our observations, and execute 5-fold cross-validation to find the optimal value of K to use.

Finally, we will fit the model and collect metrics on model performance, noting the accuracy we were able to obtain for our model.

One of the ways in which we plan on visualizing our findings is with a scatterplot, where each data point depicts one star. Whether each star is a pulsar star or not will be denoted by colour.


## Outcomes

We expect to discover which variables are most influential in the categorization of pulsar stars.

# Works Cited
Cofield, Calla. (2016, April 21). What Are Pulsars?. SPACE.COM. 
 https://www.space.com/32661-pulsars.html

Mumtaz, Asad. (2020, Jul 16). How to Find the Best Predictors for ML Algorithms. TOWARDSDATASCIENCE.COM. 
 https://towardsdatascience.com/how-to-find-the-best-predictors-for-ml-algorithms-4b28a71a8a80